In [2]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from codecarbon import EmissionsTracker
from ptflops import get_model_complexity_info
import os
import pandas as pd
import gc

In [3]:
# ----------------------------
# Clean GPU Memory
# ----------------------------
torch.cuda.empty_cache()
gc.collect()


0

In [4]:
# ----------------------------
# Device Setup
# ----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [5]:
# ----------------------------
# MobileNetV3-Large Model Definition
# ----------------------------
class MobileNetV3Large_Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.base_model = models.mobilenet_v3_large(weights="IMAGENET1K_V1")
        in_features = self.base_model.classifier[3].in_features
        self.base_model.classifier[3] = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

In [6]:
# ----------------------------
# Dataset (Memory Safe)
# ----------------------------
transform = transforms.Compose([
    transforms.Resize((160, 160)),   
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_dataset   = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)  
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:13<00:00, 12.3MB/s]


In [7]:
# ----------------------------
# Model Setup
# ----------------------------
num_classes = 10
model = MobileNetV3Large_Model(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ----------------------------
# FLOPs & Params
# ----------------------------
macs, params = get_model_complexity_info(model, (3, 64, 64),
                                         as_strings=True,
                                         print_per_layer_stat=False,
                                         verbose=False)
macs_value = macs.replace('Mac', '').strip()
if 'G' in macs_value:
    macs_num = float(macs_value.replace('G', '')) * 2
    flops_per_forward = f"{macs_num:.2f} GFLOPs"
elif 'M' in macs_value:
    macs_num = float(macs_value.replace('M', '')) / 1000 * 2
    flops_per_forward = f"{macs_num:.3f} GFLOPs"
else:
    flops_per_forward = f"{macs_value} MACs"

print(f"FLOPs per forward pass: {flops_per_forward} | Params: {params}")


Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth


100%|██████████| 21.1M/21.1M [00:00<00:00, 223MB/s]


FLOPs per forward pass: 0.043 GFLOPs | Params: 4.21 M


In [8]:
# ----------------------------
# Training & Validation
# ----------------------------
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)
    return total_loss / len(loader.dataset)

def validate(model, loader, criterion):
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    acc = 100 * correct / total
    return val_loss / total, acc

In [9]:
# ----------------------------
# Energy & CO₂ Tracking
# ----------------------------
output_dir = "./emissions_mobilenetv3_no_tuning"
os.makedirs(output_dir, exist_ok=True)

tracker = EmissionsTracker(
    project_name="MobileNetV3Large_CIFAR10_NoTuning",
    output_dir=output_dir,
    measure_power_secs=1
)
tracker.start()
start_time = time.time()

[codecarbon WARNING @ 01:47:00] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 01:47:00] [setup] RAM Tracking...
[codecarbon INFO @ 01:47:00] [setup] CPU Tracking...
[codecarbon WARNING @ 01:47:01] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 01:47:01] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 01:47:01] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 01:47:01] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 01:47:01] [setup] GPU Tracking...
[codecarbon INFO @ 01:47:01] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 01:47:01] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation mode

In [10]:
# ----------------------------
# Training Loop (4 Epochs)
# ----------------------------
for epoch in range(4):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)
    print(f"Epoch {epoch+1}/4 | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.2f}%")


[codecarbon INFO @ 01:47:25] Energy consumed for RAM : 0.000063 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 01:47:25] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 01:47:25] Energy consumed for All CPU : 0.000270 kWh
[codecarbon INFO @ 01:47:25] Energy consumed for all GPUs : 0.000186 kWh. Total GPU Power : 29.24228891159518 W
[codecarbon INFO @ 01:47:25] 0.000519 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 01:47:26] Energy consumed for RAM : 0.000066 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 01:47:26] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 01:47:26] Energy consumed for All CPU : 0.000282 kWh
[codecarbon INFO @ 01:47:26] Energy consumed for all GPUs : 0.000198 kWh. Total GPU Power : 42.05164801474334 W
[codecarbon INFO @ 01:47:26] 0.000545 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 01:47

Epoch 1/4 | Train Loss: 0.3893 | Val Acc: 88.30%


[codecarbon INFO @ 01:54:39] Energy consumed for RAM : 0.001252 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 01:54:39] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 01:54:39] Energy consumed for All CPU : 0.005331 kWh
[codecarbon INFO @ 01:54:39] Energy consumed for all GPUs : 0.007440 kWh. Total GPU Power : 52.52017501038175 W
[codecarbon INFO @ 01:54:39] 0.014022 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 01:54:40] Energy consumed for RAM : 0.001255 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 01:54:40] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 01:54:40] Energy consumed for All CPU : 0.005342 kWh
[codecarbon INFO @ 01:54:40] Energy consumed for all GPUs : 0.007458 kWh. Total GPU Power : 65.2968881297252 W
[codecarbon INFO @ 01:54:40] 0.014055 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 01:54:

Epoch 2/4 | Train Loss: 0.2145 | Val Acc: 87.12%


[codecarbon INFO @ 02:01:42] Energy consumed for RAM : 0.002409 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 02:01:42] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 02:01:42] Energy consumed for All CPU : 0.010259 kWh
[codecarbon INFO @ 02:01:42] Energy consumed for all GPUs : 0.014752 kWh. Total GPU Power : 59.13793888524488 W
[codecarbon INFO @ 02:01:42] 0.027420 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 02:01:42] 0.013505 g.CO2eq/s mean an estimation of 425.8783417221739 kg.CO2eq/year
[codecarbon INFO @ 02:01:43] Energy consumed for RAM : 0.002412 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 02:01:43] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 02:01:43] Energy consumed for All CPU : 0.010271 kWh
[codecarbon INFO @ 02:01:43] Energy consumed for all GPUs : 0.014770 kWh. Total GPU Power : 66.68269695259495 W
[codecarbon INFO @ 02:01:43] 0.027

Epoch 3/4 | Train Loss: 0.1669 | Val Acc: 88.32%


[codecarbon INFO @ 02:08:45] Energy consumed for RAM : 0.003567 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 02:08:45] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 02:08:45] Energy consumed for All CPU : 0.015188 kWh
[codecarbon INFO @ 02:08:45] Energy consumed for all GPUs : 0.022064 kWh. Total GPU Power : 54.47443513534427 W
[codecarbon INFO @ 02:08:45] 0.040818 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 02:08:46] Energy consumed for RAM : 0.003569 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 02:08:46] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 02:08:46] Energy consumed for All CPU : 0.015199 kWh
[codecarbon INFO @ 02:08:46] Energy consumed for all GPUs : 0.022082 kWh. Total GPU Power : 65.04106831959487 W
[codecarbon INFO @ 02:08:46] 0.040851 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 02:08

Epoch 4/4 | Train Loss: 0.1372 | Val Acc: 89.36%


In [11]:
# ----------------------------
# Stop Tracker & Show Results
# ----------------------------
emissions = tracker.stop()
elapsed = time.time() - start_time

csv_file = os.path.join(output_dir, "emissions.csv")
df = pd.read_csv(csv_file)
energy = df['energy_consumed'].iloc[-1]  # kWh

print("\n==============================")
print("✅ Final Results Summary (MobileNetV3-Large)")
print("==============================")
print(f"CO₂ Emissions : {emissions:.6f} kg")
print(f"Energy Used   : {energy:.6f} kWh")
print(f"Time Taken    : {elapsed/60:.2f} min")
print(f"FLOPs         : {flops_per_forward}")
print("==============================")


[codecarbon INFO @ 02:16:53] Energy consumed for RAM : 0.004901 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 02:16:53] Delta energy consumed for CPU with constant : 0.000003 kWh, power : 42.5 W
[codecarbon INFO @ 02:16:53] Energy consumed for All CPU : 0.020869 kWh
[codecarbon INFO @ 02:16:53] Energy consumed for all GPUs : 0.030056 kWh. Total GPU Power : 37.89082689953587 W
[codecarbon INFO @ 02:16:53] 0.055826 kWh of electricity and 0.000000 L of water were used since the beginning.



✅ Final Results Summary (MobileNetV3-Large)
CO₂ Emissions : 0.026282 kg
Energy Used   : 0.055826 kWh
Time Taken    : 29.86 min
FLOPs         : 0.043 GFLOPs
